In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
naver = pd.read_csv('/content/drive/MyDrive/딥러닝 공부/연구실 프로젝트 1/naver_token')
steam = pd.read_csv('/content/drive/MyDrive/딥러닝 공부/연구실 프로젝트 1/steam_token')
total = pd.read_csv('/content/drive/MyDrive/딥러닝 공부/연구실 프로젝트 1/total_token')

In [3]:
naver.head(2)

,데이터,label,tokenized
0,배공빠르고 굿,1,"['배공', '빠르', '굿']"
1,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고,0,"['택배', '엉망', '용', '저희', '집', '밑', '층', '말', '없..."


In [4]:
import numpy as np
from collections import Counter

naver_n = np.hstack(naver[naver.label == 0]['tokenized'].values)
naver_p = np.hstack(naver[naver.label == 1]['tokenized'].values)
steam_n = np.hstack(steam[steam.label == 0]['tokenized'].values)
steam_p = np.hstack(steam[steam.label == 1]['tokenized'].values)
total_n = np.hstack(total[total.label == 0]['tokenized'].values)
total_p = np.hstack(total[total.label == 1]['tokenized'].values)

naver_n_count = Counter(naver_n)
naver_p_count = Counter(naver_p)
steam_n_count = Counter(steam_n)
steam_p_count = Counter(steam_p)
total_n_count = Counter(total_n)
total_p_count = Counter(total_p)
print('네이버 토큰 단어 확인')
# 최빈 값 20개 확인해보기
print(naver_n_count.most_common(20))
print(naver_p_count.most_common(20))

print('스팀 토큰 단어 확인')
# 최빈 값 20개 확인해보기
print(steam_n_count.most_common(20))
print(steam_p_count.most_common(20))

print('전체 토큰 단어 확인')
# 최빈 값 20개 확인해보기
print(total_n_count.most_common(20))
print(total_p_count.most_common(20))

네이버 토큰 단어 확인
[("['재', '구매']", 26), ("['배송', '너무', '느려요']", 17), ("['그냥', '그래요']", 9), ("['좋', '아요']", 9), ("['배송', '빠르', '좋', '아요']", 9), ("['배송', '빨', '라요']", 7), ("['생각', '보다', '별로', '네요']", 7), ("['잘', '받', '았', '습니다']", 7), ("['그저', '그래요']", 7), ("['별로', '에요']", 7), ("['배송', '너무', '느', '립니']", 7), ("['별로']", 6), ("['사이즈', '작', '아요']", 6), ("['별루', '입니다']", 6), ("['배송', '느림']", 5), ("['냄새', '심해요']", 5), ("['배송', '너무', '늦', '네요']", 5), ("['배송', '너무', '늦', '어요']", 5), ("['별로', '예', '요']", 5), ("['배송', '겁나', '느림']", 5)]
[("['재', '구매']", 29), ("['좋', '아요']", 14), ("['좋', '습니다']", 9), ("['만족', '합니다']", 8), ("['감사', '합니다']", 8), ("['배송', '빨', '라요']", 7), ("['굿']", 6), ("['조아', '요']", 6), ("['잘', '받', '았', '습니다']", 6), ("['좋', '네요']", 5), ("['빠른', '배송', '감사', '합니다']", 5), ("['배송', '빠르', '네요']", 5), ("['배송', '빠르', '좋', '네요']", 5), ("['맘', '어요']", 4), ("['괜찮', '아요']", 4), ("['재', '구매', '빠른', '배송', '감사', '합니다']", 4), ("['맛있', '어요']", 4), ("['배송', '빠르', '좋', '아요']", 4), ("['재', '구매', '배송', '빠르

In [5]:
# 길이 비교
naver_p_len = naver[naver['label']==1]['tokenized'].map(lambda x: len(x))
steam_p_len  = steam[steam['label']==1]['tokenized'].map(lambda x: len(x))
total_p_len  = total[total['label']==1]['tokenized'].map(lambda x: len(x))
print('긍정 리뷰의 평균 길이 정도')
print('네이버 : ', np.mean(naver_p_len))
print('스팀 : ', np.mean(steam_p_len))
print('전체 : ', np.mean(total_p_len))

naver_n_len = naver[naver['label']==0]['tokenized'].map(lambda x: len(x))
steam_n_len  = steam[steam['label']==0]['tokenized'].map(lambda x: len(x))
total_n_len  = total[total['label']==0]['tokenized'].map(lambda x: len(x))
print('부정 리뷰의 평균 길이 정도')
print('네이버 : ', np.mean(naver_n_len))
print('스팀 : ', np.mean(steam_n_len))
print('전체 : ', np.mean(total_n_len))


긍정 리뷰의 평균 길이 정도
네이버 :  77.66683006862883
스팀 :  92.04580366429315
전체 :  82.46137122965196
부정 리뷰의 평균 길이 정도
네이버 :  97.6548319327731
스팀 :  93.83281003139811
전체 :  96.38410600442796


# 정수 인코딩
* 컴퓨터가 학습을 잘하기 위해서 텍스트보다는 숫자를 훨씬 더 잘 처리하기 때문에
위에 토큰 단어마다의 고유의 정수를 매핑하는 정수 인코딩을 사용


In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 학습 및 test 데이터 분리
naver_train, naver_test = train_test_split(naver, test_size = 0.25, random_state = 42)
steam_train, steam_test = train_test_split(steam, test_size = 0.25, random_state = 42)
total_train, total_test = train_test_split(total, test_size = 0.25, random_state = 42)

# 토큰화 단어만 분리
n_train = naver_train['tokenized'].values
s_train = steam_train['tokenized'].values
t_train = total_train['tokenized'].values
n_test = naver_test['tokenized'].values
s_test = steam_test['tokenized'].values
t_test = total_test['tokenized'].values

# 정답 분리
n_y_train = naver_train['label'].values
s_y_train = steam_train['label'].values
t_y_train = total_train['label'].values

n_y_test = naver_test['label'].values
s_y_test = steam_test['label'].values
t_y_test = total_test['label'].values

In [7]:
# 토큰나이저 사용
tokenizer = Tokenizer()
tokenizer.fit_on_texts(n_train)

In [8]:
threshold = 2 # 1번 이하인 빈도수 팍악을 위해서 사용
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 빈도수

for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 40168
등장 빈도가 1번 이하인 희귀 단어의 수: 18302
단어 집합에서 희귀 단어의 비율: 45.56363274248158
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 0.7972230050803256


* 전체 희귀 단어의 비율은 높지만, 전체 등장 빈도에서 희귀 단어의 등장 빈도 비율이 1%도 안되므로 이를 학습과정에서 제외

In [9]:
vocab_size = total_cnt - rare_cnt + 2
print('단어 집합의 크기 :',vocab_size) # 단어의 집합의 크기가 많이 감소됨.

단어 집합의 크기 : 21868


* oov 토큰 사용.
  * oov = out of vocabulary의 의미로 train에서 빈도수가 낮은 단어들을 무시하지는 않지만, 특정 값으로 토큰화 한다.

In [10]:
tokenizer = Tokenizer(vocab_size, oov_token = 'OOV')
tokenizer.fit_on_texts(n_train)
# 숫자로 변환
naver_X_train = tokenizer.texts_to_sequences(n_train)
naver_X_test = tokenizer.texts_to_sequences(n_test)

In [11]:
# 정수 인코딩 결과의 확인
print(naver_X_train[:2])

[[21, 9, 729, 244, 1682, 30, 3, 6, 116, 437, 492], [21, 9, 11, 106, 72, 482, 45, 222, 326, 131, 30, 21, 9, 14, 4, 544, 29, 40, 179, 5873, 14334, 7, 58, 33, 12, 100, 701, 59, 740]]


# 패딩
* 리뷰들의 길이가 서로 다르므로 이를 반영할 수 있도록 리뷰의 최대 길이를 확인한다.

In [12]:
# 최대 길이 및 평균 길이 파악
print('리뷰의 최대 길이 :',max(len(review) for review in n_train))
print('리뷰의 평균 길이 :',sum(map(len, n_train))/len(n_train))

리뷰의 최대 길이 : 426
리뷰의 평균 길이 : 87.66183355787058


In [13]:
# 최대 길이 만큼의 길이를 늘려줌.
max_len = 426
naver_X_train = pad_sequences(naver_X_train, maxlen=max_len)
naver_X_test = pad_sequences(naver_X_test, maxlen=max_len)

In [14]:
# 결과 확인
naver_X_test[:1]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 

# 스팀 및 전체 데이터에 대해서도 진행.

In [15]:
# 스팀
tokenizer = Tokenizer()
tokenizer.fit_on_texts(s_train)

threshold = 2 # 1번 이하인 빈도수 팍악을 위해서 사용
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 빈도수

for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 32993
등장 빈도가 1번 이하인 희귀 단어의 수: 13998
단어 집합에서 희귀 단어의 비율: 42.42718152335344
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 1.152022912071633


In [16]:
# 1% 대이므로 제거
vocab_size = total_cnt - rare_cnt + 2
print('단어 집합의 크기 :',vocab_size) # 단어의 집합의 크기가 많이 감소됨.

단어 집합의 크기 : 18997


In [17]:
tokenizer = Tokenizer(vocab_size, oov_token = 'OOV')
tokenizer.fit_on_texts(s_train)
# 숫자로 변환
steam_X_train = tokenizer.texts_to_sequences(s_train)
steam_X_test = tokenizer.texts_to_sequences(s_test)

In [18]:
# 최대 길이 및 평균 길이 파악
print('리뷰의 최대 길이 :',max(len(review) for review in s_train))
print('리뷰의 평균 길이 :',sum(map(len, n_train))/len(s_train))

리뷰의 최대 길이 : 411
리뷰의 평균 길이 : 175.25353671382285


In [19]:
# 최대 길이 만큼의 길이를 늘려줌.
max_len = 411
steam_X_train = pad_sequences(steam_X_train, maxlen=max_len)
steam_X_test = pad_sequences(steam_X_test, maxlen=max_len)

In [20]:
# 확인
steam_X_train[:1]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 

In [21]:
# 전체
tokenizer = Tokenizer()
tokenizer.fit_on_texts(t_train)

threshold = 2 # 1번 이하인 빈도수 팍악을 위해서 사용
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 빈도수

for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 56234
등장 빈도가 1번 이하인 희귀 단어의 수: 24380
단어 집합에서 희귀 단어의 비율: 43.354554184301314
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 0.6953037904323575


In [22]:
# 1% 미만 대이므로 제거
vocab_size = total_cnt - rare_cnt + 2
print('단어 집합의 크기 :',vocab_size) # 단어의 집합의 크기가 많이 감소됨.

단어 집합의 크기 : 31856


In [23]:
tokenizer = Tokenizer(vocab_size, oov_token = 'OOV')
tokenizer.fit_on_texts(t_train)
# 숫자로 변환
total_X_train = tokenizer.texts_to_sequences(t_train)
total_X_test = tokenizer.texts_to_sequences(t_test)

In [24]:
# 최대 길이 및 평균 길이 파악
print('리뷰의 최대 길이 :',max(len(review) for review in t_train))
print('리뷰의 평균 길이 :',sum(map(len, n_train))/len(t_train))

리뷰의 최대 길이 : 426
리뷰의 평균 길이 : 58.43498672926364


In [25]:
# 최대 길이 만큼의 길이를 늘려줌.
max_len = 426
total_X_train = pad_sequences(total_X_train, maxlen=max_len)
total_X_test = pad_sequences(total_X_test, maxlen=max_len)

In [26]:
# 확인
total_X_train[:1]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 

# 학습
* naver의 단어 크기 = 21868
* team의 단어 크기 = 18997
* total의 단어 크기 = 31856
이므로 이를 각각

In [66]:
from tensorflow.keras.layers import Embedding, Dense, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import AdamW

naver_vocab, steam_vocab, total_vocab = 21868, 18997, 31856

# 모델 설계
model_naver = Sequential()
model_naver.add(Embedding(naver_vocab, 100))
model_naver.add(GRU(100))
model_naver.add(Dense(1, activation = 'relu'))

model_steam = Sequential()
model_steam.add(Embedding(steam_vocab, 100))
model_steam.add(GRU(100))
model_steam.add(Dense(1, activation = 'relu'))

model_total = Sequential()
model_total.add(Embedding(total_vocab, 100))
model_total.add(GRU(100))
model_total.add(Dense(1, activation = 'relu'))

early_stop1 = EarlyStopping(monitor = 'val_loss', mode = 'min', patience = 5, verbose = 1)
early_stop2 = EarlyStopping(monitor = 'val_loss', mode = 'min', patience = 5, verbose = 1)
early_stop3 = EarlyStopping(monitor = 'val_loss', mode = 'min', patience = 5, verbose = 1)
checkpoint_n = ModelCheckpoint('naver_best.h5', monitor = 'val_acc', mode = 'max', verbose =1, save_best_only = True)
checkpoint_s = ModelCheckpoint('steam_best.h5', monitor = 'val_acc', mode = 'max', verbose =1, save_best_only = True)
checkpoint_t= ModelCheckpoint('total_best.h5', monitor = 'val_acc', mode = 'max', verbose =1, save_best_only = True)

In [61]:
# 학습 진행.
optimizer_adamw = AdamW(weight_decay=1e-4)

model_naver.compile(optimizer=optimizer_adamw, loss='binary_crossentropy', metrics = ['acc'])
history_naver = model_naver.fit(naver_X_train, n_y_train, epochs = 100, callbacks = [early_stop1, checkpoint_n], batch_size = 64, validation_split = 0.15)

Epoch 1/100
1992/1992 [==============================] - ETA: 0s - loss: 0.4569 - acc: 0.8590
Epoch 1: val_acc improved from -inf to 0.85674, saving model to naver_best.h5
1992/1992 [==============================] - 79s 38ms/step - loss: 0.4569 - acc: 0.8590 - val_loss: 0.3926 - val_acc: 0.8567
Epoch 2/100
1992/1992 [==============================] - ETA: 0s - loss: 0.3895 - acc: 0.8786
Epoch 2: val_acc did not improve from 0.85674
1992/1992 [==============================] - 43s 22ms/step - loss: 0.3895 - acc: 0.8786 - val_loss: 0.4415 - val_acc: 0.8469
Epoch 3/100
1992/1992 [==============================] - ETA: 0s - loss: 0.3491 - acc: 0.9018
Epoch 3: val_acc improved from 0.85674 to 0.86275, saving model to naver_best.h5
1992/1992 [==============================] - 41s 21ms/step - loss: 0.3491 - acc: 0.9018 - val_loss: 0.3899 - val_acc: 0.8627
Epoch 4/100
1992/1992 [==============================] - ETA: 0s - loss: 0.3424 - acc: 0.9003
Epoch 4: val_acc improved from 0.86275 to 0.

In [45]:
optimizer_adamw = AdamW(weight_decay=1e-4)
model_steam.compile(optimizer=optimizer_adamw, loss='binary_crossentropy', metrics = ['acc'])
history_steam = model_steam.fit(steam_X_train, s_y_train, epochs = 100, callbacks = [early_stop2, checkpoint_s], batch_size = 64, validation_split = 0.15)

Epoch 1/100
997/997 [==============================] - ETA: 0s - loss: 0.6564 - acc: 0.6746
Epoch 1: val_acc improved from -inf to 0.74578, saving model to steam_best.h5
997/997 [==============================] - 59s 56ms/step - loss: 0.6564 - acc: 0.6746 - val_loss: 0.5591 - val_acc: 0.7458
Epoch 2/100
997/997 [==============================] - ETA: 0s - loss: 0.5243 - acc: 0.7686
Epoch 2: val_acc improved from 0.74578 to 0.75573, saving model to steam_best.h5
997/997 [==============================] - 25s 25ms/step - loss: 0.5243 - acc: 0.7686 - val_loss: 0.5824 - val_acc: 0.7557
Epoch 3/100
997/997 [==============================] - ETA: 0s - loss: 0.4607 - acc: 0.8111
Epoch 3: val_acc did not improve from 0.75573
997/997 [==============================] - 24s 24ms/step - loss: 0.4607 - acc: 0.8111 - val_loss: 0.5436 - val_acc: 0.7512
Epoch 4/100
997/997 [==============================] - ETA: 0s - loss: 0.4212 - acc: 0.8340
Epoch 4: val_acc did not improve from 0.75573
997/997 [===

In [62]:
optimizer_adamw = AdamW(weight_decay=1e-4)
model_total.compile(optimizer=optimizer_adamw, loss='binary_crossentropy', metrics = ['acc'])
history_total = model_naver.fit(total_X_train, t_y_train, epochs = 100, callbacks = [early_stop3, checkpoint_t], batch_size = 64, validation_split = 0.15)

Epoch 1/100
2988/2988 [==============================] - ETA: 0s - loss: 0.6618 - acc: 0.7072
Epoch 1: val_acc improved from -inf to 0.81236, saving model to total_best.h5
2988/2988 [==============================] - 65s 22ms/step - loss: 0.6618 - acc: 0.7072 - val_loss: 0.4587 - val_acc: 0.8124
Epoch 2/100
2988/2988 [==============================] - ETA: 0s - loss: 0.4277 - acc: 0.8389
Epoch 2: val_acc improved from 0.81236 to 0.82270, saving model to total_best.h5
2988/2988 [==============================] - 63s 21ms/step - loss: 0.4277 - acc: 0.8389 - val_loss: 0.4396 - val_acc: 0.8227
Epoch 3/100
2988/2988 [==============================] - ETA: 0s - loss: 0.3821 - acc: 0.8612
Epoch 3: val_acc improved from 0.82270 to 0.85904, saving model to total_best.h5
2988/2988 [==============================] - 61s 20ms/step - loss: 0.3821 - acc: 0.8612 - val_loss: 0.3991 - val_acc: 0.8590
Epoch 4/100
2988/2988 [==============================] - ETA: 0s - loss: 0.4603 - acc: 0.8554
Epoch 4: 

In [ ]:
!pip install konlpy

In [ ]:
!pip install mecab-python

In [ ]:
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

# 결과 확인

In [76]:
import re
from konlpy.tag import Mecab
from tensorflow.keras.models import load_model

model_n = load_model('naver_best.h5')
model_s = load_model('/content/steam_best.h5')
model_t = load_model('/content/total_best.h5')
mecab = Mecab()
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']

def predict_sentence(input_sen):
  input_sen = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣]', '', input_sen)
  input_sen = mecab.morphs(input_sen)
  input_sen = [word for word in input_sen if not word in stopwords]
  encoded = tokenizer.texts_to_sequences([input_sen])
  pad_sen = pad_sequences(encoded, maxlen=426)
  score_n = float(model_n.predict(pad_sen))
  score_s = float(model_s.predict(pad_sen))
  score_t = float(model_t.predict(pad_sen))
  return [score_n, score_s, score_t]


In [77]:
while 1:
  input_sen = input()
  result = predict_sentence(input_sen)
  print('네이버 : ',  result[0])
  print('스팀 : ',  result[1])
  print('전체 : ',  result[2])


정말 좋네요
1/1 [==============================] - 0s 451ms/step
네이버 :  0.45072153210639954
스팀 :  0.5288102030754089
전체 :  0.5858303308486938
별로에요
1/1 [==============================] - 0s 23ms/step
네이버 :  0.43837353587150574
스팀 :  0.49719059467315674
전체 :  0.5538169145584106
쓰레기
1/1 [==============================] - 0s 24ms/step
네이버 :  0.4050372540950775
스팀 :  0.44072240591049194
전체 :  0.5077041387557983


KeyboardInterrupt: ignored

* AdamW 대신 RMSprp 사용


In [67]:
# 학습 진행.
model_naver.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics = ['acc'])
history_naver = model_naver.fit(naver_X_train, n_y_train, epochs = 100, callbacks = [early_stop1, checkpoint_n], batch_size = 64, validation_split = 0.15)

model_steam.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics = ['acc'])
history_steam = model_steam.fit(steam_X_train, s_y_train, epochs = 100, callbacks = [early_stop2, checkpoint_s], batch_size = 64, validation_split = 0.15)

model_total.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics = ['acc'])
history_total = model_naver.fit(total_X_train, t_y_train, epochs = 100, callbacks = [early_stop3, checkpoint_t], batch_size = 64, validation_split = 0.15)

Epoch 1/100
1992/1992 [==============================] - ETA: 0s - loss: 0.4880 - acc: 0.8677
Epoch 1: val_acc improved from -inf to 0.89667, saving model to naver_best.h5
1992/1992 [==============================] - 73s 36ms/step - loss: 0.4880 - acc: 0.8677 - val_loss: 0.4768 - val_acc: 0.8967
Epoch 2/100
1992/1992 [==============================] - ETA: 0s - loss: 0.4144 - acc: 0.9022
Epoch 2: val_acc improved from 0.89667 to 0.90787, saving model to naver_best.h5
1992/1992 [==============================] - 41s 20ms/step - loss: 0.4144 - acc: 0.9022 - val_loss: 0.4457 - val_acc: 0.9079
Epoch 3/100
1992/1992 [==============================] - ETA: 0s - loss: 0.3792 - acc: 0.9143
Epoch 3: val_acc improved from 0.90787 to 0.91428, saving model to naver_best.h5
1992/1992 [==============================] - 40s 20ms/step - loss: 0.3792 - acc: 0.9143 - val_loss: 0.3734 - val_acc: 0.9143
Epoch 4/100
1991/1992 [============================>.] - ETA: 0s - loss: 0.3275 - acc: 0.9216
Epoch 4: 

In [72]:
model_n = load_model('naver_best2.h5')
model_s = load_model('steam_best2.h5')
model_t = load_model('total_best2.h5')
mecab = Mecab()
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']

def predict_sentence(input_sen):
  input_sen = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣]', '', input_sen)
  input_sen = mecab.morphs(input_sen)
  input_sen = [word for word in input_sen if not word in stopwords]
  encoded = tokenizer.texts_to_sequences([input_sen])
  pad_sen = pad_sequences(encoded, maxlen=426)
  score_n = float(model_n.predict(pad_sen))
  score_s = float(model_s.predict(pad_sen))
  score_t = float(model_t.predict(pad_sen))
  return [score_n, score_s, score_t]

for i in range(3):
  input_sen = input()
  result = predict_sentence(input_sen)
  print('네이버 : ',  result[0])
  print('스팀 : ',  result[1])
  print('전체 : ',  result[2])

정말 좋네요
1/1 [==============================] - 0s 36ms/step
네이버 :  0.4895153045654297
스팀 :  0.8196146488189697
전체 :  0.5487467646598816
별로에요
1/1 [==============================] - 0s 22ms/step
네이버 :  0.4270845055580139
스팀 :  0.8182191848754883
전체 :  0.5511476993560791
쓰레기
1/1 [==============================] - 0s 25ms/step
네이버 :  0.35353827476501465
스팀 :  0.7850578427314758
전체 :  0.5221859216690063
